# Does not work properly -> Only linear forecast ⚠️ 

# Fine Grained Demand Forecasting

*Prerequisite: Make sure to run 01_Introduction_And_Setup before running this notebook.*

In this notebook we first find an appropriate time series model and then apply that very same approach to train multiple models in parallel with great speed and cost-effectiveness.  

Key highlights for this notebook:
- Use Databricks' collaborative and interactive notebook environment to find an appropriate time series mdoel
- Pandas UDFs (user-defined functions) can take your single-node data science code, and distribute it across different keys (e.g. SKU)  
- Hyperopt can also perform hyperparameter tuning from within a Pandas UDF  

In [0]:
#If True, all output files are in user specific databases, If False, a global database for the report is used
#user_based_data = True

In [0]:
# %run ./_resources_outside/00-global-setup $reset_all_data=false $db_prefix=demand_level_forecasting

In [0]:
dbutils.widgets.dropdown('reset_all_data', 'false', ['true', 'false'], 'Reset all data')
dbutils.widgets.text('catalogName',  'maxkoehler_demos' , 'Catalog Name')
dbutils.widgets.text('dbName',  'demand_db' , 'Database Name')

In [0]:
catalogName = dbutils.widgets.get('catalogName')
dbName = dbutils.widgets.get('dbName')
reset_all_data = dbutils.widgets.get('reset_all_data') == 'true'

In [0]:
%run ./_resources/00-setup $reset_all_data=false $catalogName=$catalogName $dbName=$dbName 


## Using Databricks SQL AI Forecast Function
ai_forecast() is a table-valued function designed to extrapolate time series data into the future

[AI_Forecast Reference](http://docs.databricks.com/aws/en/sql/language-manual/functions/ai_forecast)


####Requirement
- Pro or Serverless SQL warehouse
- In Databricks Runtime 15.1 and above, this function is supported in Databricks notebooks, including notebooks that are run as a task in a Databricks workflow.
- For batch inference workloads, Databricks recommends Databricks Runtime 15.4 ML LTS for improved performance.

Comment Patrick: Only worked for me with using SQL Serverless Warehouse. With normal Serverless failed with FEATURE NOT ENABLED error

###Check AI_Forecast() Function Capabilities
Using `AI_Forecast()` Function to forecast historical data for one SKU to show forecast quality 

In [0]:
%sql
WITH input_data AS (
  SELECT Date, Demand, SKU FROM ${catalogName}.${dbName}.part_level_demand
  WHERE SKU = (SELECT SKU FROM ${catalogName}.${dbName}.part_level_demand LIMIT 1) 
  AND Date <= (SELECT DATE_ADD(MIN(Date), 365*2) FROM ${catalogName}.${dbName}.part_level_demand)
)
SELECT forecast.Date, forecast.SKU, Demand, Demand_Forecast FROM AI_FORECAST(
    TABLE(input_data),
    horizon => DATE_ADD(DAY, 40, (SELECT MAX(Date) FROM ${catalogName}.${dbName}.part_level_demand)) ,
    time_col => 'Date',
    value_col => 'Demand',
    group_col => 'SKU',
    frequency => 'week',
    parameters => '{ "global_floor": 0, "yearly_order": 12 }'
) AS forecast
LEFT JOIN ${catalogName}.${dbName}.part_level_demand AS demand
ON forecast.Date = Demand.Date AND forecast.SKU = demand.SKU 
ORDER BY Date ASC

Databricks visualization. Run in Databricks to view.

## Make demand forecasts using AI_Forecast() function

Apply the `AI_Forecast()` function to our whole dataset to generate demand forecasts for the next 40 days and store the results into a delta table.

In [0]:
%sql
CREATE OR REPLACE TABLE ${catalogName}.${dbName}.part_level_demand_forecasts AS (
SELECT forecast.Date, forecast.SKU, demand.Product, forecast.Demand_Forecast AS Demand, TRUE AS is_forecast FROM AI_FORECAST(
    TABLE(SELECT Date, Demand, SKU FROM ${catalogName}.${dbName}.part_level_demand),
    horizon => DATE_ADD(DAY, 90, (SELECT MAX(Date) FROM ${catalogName}.${dbName}.part_level_demand)),
    time_col => 'Date',
    value_col => 'Demand',
    group_col => 'SKU',
    frequency => 'week',
    parameters => '{ "global_floor": 0, "yearly_order": 12 }'
) as forecast
LEFT JOIN (SELECT DISTINCT SKU, Product FROM ${catalogName}.${dbName}.part_level_demand) AS demand
ON forecast.SKU = demand.SKU
UNION ALL 
SELECT Date, SKU, Product, Demand, FALSE as is_forecast FROM ${catalogName}.${dbName}.part_level_demand
ORDER BY SKU, Date
)



In [0]:
%sql
SELECT * FROM ${catalogName}.${dbName}.part_level_demand_forecasts